In [1]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.2.0'

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1683214368817617857
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17754445555355964052
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3055235892
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3338104938171009113
physical_device_desc: "device: 0, name: GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17299166647373436299
physical_device_desc: "device: XLA_GPU device"
]


In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    r'D:\Python37\Projects\Nvidia_AI_Hackthon\dataset\training',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
   r'D:\Python37\Projects\Nvidia_AI_Hackthon\dataset\training', # same directory as training data
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # set as validation data



# Preprocessing the Test set
#test_datagen = ImageDataGenerator(rescale = 1./255)

#test_generator  = test_datagen.flow_from_directory(r'D:\Python37\Projects\Nvidia_AI_Hackthon\dataset\test',
 #                                           target_size = (64, 64),
  #                                          batch_size = 32,
   #                                         class_mode = 'categorical')


Found 4700 images belonging to 2 classes.
Found 1175 images belonging to 2 classes.


In [7]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=2, activation='softmax'))

In [8]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(r"D:\Python37\Projects\Nvidia_AI_Hackthon\models\model1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 3,
                          restore_best_weights = True)

learning_rate= ReduceLROnPlateau(monitor = 'val_loss',
                                 patience = 3,
                                 verbose = 3)


# we put our call backs into a callback list
callbacks = [checkpoint, learning_rate]


In [10]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [14]:
# Training the CNN on the Training set and evaluating it on the Test set
history=cnn.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples //32,
    callbacks=callbacks,
    epochs = 10)

Epoch 1/10
146/146 [==============================] - ETA: 0s - loss: 0.1137 - accuracy: 0.9587
Epoch 00001: val_loss did not improve from 0.14952
146/146 [==============================] - 9s 63ms/step - loss: 0.1137 - accuracy: 0.9587 - val_loss: 0.1577 - val_accuracy: 0.9358 - lr: 1.0000e-04
Epoch 2/10
146/146 [==============================] - ETA: 0s - loss: 0.1169 - accuracy: 0.9565
Epoch 00002: val_loss did not improve from 0.14952
146/146 [==============================] - 9s 64ms/step - loss: 0.1169 - accuracy: 0.9565 - val_loss: 0.1747 - val_accuracy: 0.9271 - lr: 1.0000e-04
Epoch 3/10
146/146 [==============================] - ETA: 0s - loss: 0.1036 - accuracy: 0.9604
Epoch 00003: val_loss did not improve from 0.14952
146/146 [==============================] - 9s 64ms/step - loss: 0.1036 - accuracy: 0.9604 - val_loss: 0.1604 - val_accuracy: 0.9271 - lr: 1.0000e-04
Epoch 4/10
145/146 [============================>.] - ETA: 0s - loss: 0.1033 - accuracy: 0.9592
Epoch 00004: val

In [15]:
loss_tr, acc_tr = cnn.evaluate_generator(train_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [16]:
print(loss_tr)
print(acc_tr)

0.10035137832164764
0.9631915092468262


In [18]:
loss_val, acc_val= cnn.evaluate_generator(validation_generator)

In [19]:
print(loss_val)
print(acc_val)

0.1611202359199524
0.9429787397384644


In [21]:
import glob
from tensorflow.keras.preprocessing import image
import numpy as np
k = glob.glob(r"D:\Python37\Projects\Nvidia_AI_Hackthon\dataset\test\*.jpg")

y_true = []
y_pred = []
for i in k: 
    test_image = image.load_img(i, target_size = (64,64,3))
    test_image = image.img_to_array(test_image)
    test_image=test_image/255
    test_image = np.expand_dims(test_image, axis = 0)
    y_pred.append(cnn.predict(test_image))
    #y_true.append(0)



In [84]:
labels=[]
for i in y_pred:
    labels.append(int(np.argmax(i[0])))

In [85]:
import random
import json
# from utils.io import write_json

def write_json(filename, result):
    with open(filename, 'w') as outfile:
        json.dump(result, outfile)

def read_json(filename):
    with open(filename, 'r') as outfile:
        data =  json.load(outfile)
    return data

def generate_sample_file(filename):
    res = {}
    for i in range(1,100):
        test_set = str(i) + '.png'
        res[test_set] = labels

    write_json(filename, res)

if __name__ == '__main__':
    generate_sample_file(r'D:\Python37\Projects\Nvidia_AI_Hackthon\results.json')
